# Dagstuhl Choir Set: Parsing and Visualization Demo

### Table of Contents
1. [DCS Content Parser](#one)
2. [Multitrack Audio Playback](#two)
3. [Beat Annotations](#three)
4. [Time-Aligned Score Representation](#four)
5. [Fundamental Frequency Trajectories](#five)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pywebaudioplayer import pywebaudioplayer as pwa
import IPython.display as ipd
import pandas as pd
import librosa
import pretty_midi

import choirset_toolbox as ct

### 1. DCS Content Parser <a class="anchor" id="one"></a>

In [ ]:
# Settings
DCS_path = '../../data/2019_DagstuhlRecording/Dagstuhl Dataset/'
song_id = 'DLI'
setting = 'QuartetA'
take = 'Take4'
part = '*'
mic = '*'

# generate table of available audio and annotation files
DCS_table = ct.DCS_content_parser(DCS_path, song_id, setting, take, part, mic)
DCS_table

### 2. Multitrack Audio Playback<a class="anchor" id="two"></a>

In [ ]:
# using pywebaudioplayer by Johan Pauwels
tracks = []
for i in range(DCS_table.shape[0]):
    tracks.append({'title': '%s (%s)' %(DCS_table.iloc[i]['Section'], DCS_table.iloc[i]['Microphone']), 
                   'path': DCS_table.iloc[i]['Audio'], 'mimetype': 'audio/wav'})
    
# Display trackswitch.js (Please re-execute cell if JavaScript error message is shown)
ts1 = pwa.trackswitch(tracks, repeat=False, globalsolo=False, onlyradiosolo=True)
ipd.HTML(ts1)

### 3. Beat Annotations<a class="anchor" id="three"></a>

In [ ]:
fn_beat = DCS_table.iloc[1, :]['BeatAnnotation']
beat_anno = ct.read_csv(fn_beat, header=None, names=['Time (sec)', 'Measure.RelativeBeatPosition'])
beat_anno

The last row of the beat annotation includes the end of the audio file in seconds and the end of the last measure encoded as .999.

In [ ]:
# sonify beat annotation using librosa
fn_mix = DCS_table.iloc[1, :]['Audio']
x_mix, Fs = librosa.load(fn_mix)
beat_soni = librosa.clicks(beat_anno.iloc[:, 0].values, sr=Fs, click_freq=1000, length=len(x_mix))
ipd.Audio(x_mix+beat_soni, rate=Fs)

### 4. Time-Aligned Score Representation<a class="anchor" id="four"></a>

In [ ]:
# to be replaced by real link...
# MIDI files or CSVs?
fn_midi = '../../data/2019_DagstuhlRecording/Dagstuhl Dataset/Annotations/SheetMusic_aligned_AudioLabsBeats/Bruckner_WAB023_BrianMarble-CPDL_warpedTo_DLI_QuartetA_Take4_Stereo_STM.mid'

patt = pretty_midi.PrettyMIDI(fn_midi)
headers = ['Onset (sec)', 'Offset (sec)', 'MIDIPitch']
sop_midi = pd.DataFrame(columns=headers)
alt_midi = pd.DataFrame(columns=headers)
ten_midi = pd.DataFrame(columns=headers)
bas_midi = pd.DataFrame(columns=headers)

for note in patt.instruments[0].notes:
    sop_midi = sop_midi.append(pd.Series([note.start, note.end, note.pitch], index=sop_midi.columns), ignore_index=True)
for note in patt.instruments[1].notes:
    alt_midi = alt_midi.append(pd.Series([note.start, note.end, note.pitch], index=alt_midi.columns), ignore_index=True)
for note in patt.instruments[2].notes:
    ten_midi = ten_midi.append(pd.Series([note.start, note.end, note.pitch], index=ten_midi.columns), ignore_index=True)
for note in patt.instruments[3].notes:
    bas_midi = bas_midi.append(pd.Series([note.start, note.end, note.pitch], index=bas_midi.columns), ignore_index=True)
    
bas_midi    

In [ ]:
# MIDI stereo sonification
ipd.Audio('../../data/2019_DagstuhlRecording/Dagstuhl Dataset/Annotations/SheetMusic_aligned_AudioLabsBeats/DLI_QuartetA_Take4_Stereo_STM_MIDI_soniStereo.wav', rate=Fs)

### 5. Fundamental Frequency Trajectories<a class="anchor" id="five"></a>

In [ ]:
fn_f0 = DCS_table.iloc[4, :]['F0CREPE']
f0 = ct.read_csv(fn_f0, header=None, names=['Time (sec)', 'F0 (Hz)', 'Confidence'])
f0

In [ ]:
# stereo sonification
x_lrx, Fs = librosa.load(DCS_table.iloc[4, :]['Audio'])
x_f0 = ct.sonify_trajectory_with_sinusoid(f0.iloc[:, 0:2].values, len(x_lrx), Fs=Fs, amplitude=0.3, smooth_len=11)
x_soni_stereo = np.vstack((x_lrx.reshape(1,-1), x_f0.reshape(1,-1)))
ipd.Audio(x_soni_stereo, rate=Fs)